In [1]:
import os
import random
import json
from shutil import copyfile
import sys

In [2]:
LENGTH = {'상의':'소매기장', '하의':'기장'}
CATEGORY_NUMBER = {'탑':0, '티셔츠':1, '니트웨어':2, '셔츠':3, '후드티':4,
                  '청바지':0, '팬츠':1, '조거팬츠':2}
LENGTH_NUMBER = {'민소매':0, '반팔':0, '캡':0, '7부소매':1, '긴팔':1,
                '미니':0, '니렝스':0, '미디':1, '발목':1, '맥시':1}
NUMBER_OF_CATEGORY = {'상의':10, '하의':6}
NUMBER_OF_LENGTH = 2

# Read Directory List

In [3]:
def readDirList(path_dir='.'):
    if os.path.isdir(path_dir):
        file_list = os.listdir(path_dir)
    else:
        file_list = None
    
    return file_list

# Parse Data List

In [4]:
def parseDataList(data_list):
    name_list = []
    
    for data in data_list:
        name_list.append(os.path.splitext(data)[0])
        
    return name_list

# Load Json Data

In [5]:
def loadJsonData(fileName):
    with open(fileName, 'r', encoding='UTF8') as f:
        json_data = json.load(f)
    
    return json_data

# Get  Rectangle Coordinate

In [6]:
def getRectCoor(rect_list):
    rect_coor = []
    
    # rect_list에 있는 모든 rect들에 대해서
    for rect in rect_list:
        # 정보가 없으면 break
        if not rect.get('X좌표'): break
        
        # x, y, w, h로 [x1,y1,x2,y2] 데이터 append
        x = rect['X좌표']
        y = rect['Y좌표']
        w = rect['가로']
        h = rect['세로']
        rect_coor.append([x, y, x+w, y+h])
    
    # 사각형 좌표 정보 리턴
    return rect_coor

# Get Category

In [7]:
def getCategory(label_list, category_type):
    category_list = []
    
    #print(label_list)
    # label_list에 있는 모든 label들에 대해서
    for label in label_list:
        # 정보가 없으면 break
        #if not label.get('카테고리'): break
        
        try:
            category = label['카테고리']
            length = label[LENGTH[category_type]]

            c_num = CATEGORY_NUMBER[category]
            l_num = LENGTH_NUMBER[length]

            category_number = c_num*NUMBER_OF_LENGTH + l_num
        except KeyError:
            category_number = -1
        
        category_list.append(category_number)
    
    return category_list

# Save List to Text File

In [8]:
def saveList2(fileName, data_list):
    f = open(fileName, 'w')
    
    for data in data_list:
        for d in data:
            f.write(str(d))
            f.write(' ')
        f.write('\n')
    
    f.close()

# Extract Data

In [26]:
'''
img_dir : img 데이터 direcotry
label_dir : label 데이터(.json) direcotry
res_img_dir : 추출한 img 데이터를 저장할 directory
res_label_dir : 추출한 label 데이터를 저장할 directory
category_type : 상의 / 하의
npc : number per category

약 category 개수 * npc 만큼의 데이터가 추출된다.
(한 이미지에 여러 category가 있으면 그만큼 데이터가 추출되는 데이터가 적어짐)
'''
def extractData(img_dir, label_dir, res_img_dir, res_label_dir, category_type, npc):
    if not os.path.exists(res_img_dir):
        os.makedirs(res_img_dir)
    if not os.path.exists(res_label_dir):
        os.makedirs(res_label_dir)
    
    # label 리스트 가져오기
    label_list = readDirList(label_dir)
    # name.json > name
    name_list = parseDataList(label_list)
    # name 리스트 무작위로 섞기
    random.shuffle(name_list)
    
    category_count = [0] * NUMBER_OF_CATEGORY[category_type]
    end_count = [npc] * NUMBER_OF_CATEGORY[category_type]

    for name in name_list:
        # label_dir/name.json 읽기
        json_data = loadJsonData(label_dir+name+'.json')

        # json 파일에서 category_type에 해당하는 사각형 좌표 가져오기
        rect_coor = getRectCoor(json_data['데이터셋 정보']['데이터셋 상세설명']['렉트좌표'][category_type])

        # json 파일에서 category_type에 해당하는 카테고리 가져오기
        category = getCategory(json_data['데이터셋 정보']['데이터셋 상세설명']['라벨링'][category_type], category_type)
        
        # 유효한 데이터만 추출
        final_list = []
        flag = False
        for i in range(len(rect_coor)):
            if category[i] == -1: continue
            if category_count[category[i]]+1 > npc: flag = True

            final_list.append([category[i]] + rect_coor[i])

        # 유효한 데이터가 없거나 해당 카테고리 이미지가 너무 많으면 continue
        if len(final_list) == 0 or flag: continue

        # 이미지 복사
        copyfile(img_dir+name+'.jpg', res_img_dir+name+'.jpg')
        # label 저장
        saveList2(res_label_dir+name+'.txt', final_list)

        # 카테고리 카우팅
        for c in category:
            if category[i] == -1: continue
            category_count[category[i]] += 1

        print(category_count, name)
        
        # 모든 카테고리를 다 추출했으면 종료
        if category_count == end_count:
            break
    
    print('finish')

In [27]:
extractData('Training/원천데이터_merged/', 'Training/라벨링데이터_merged/', 'Training/원천데이터_상의_100/', 'Training/라벨링데이터_상의_100/', '상의', 100)
extractData('Validation/원천데이터_merged/', 'Validation/라벨링데이터_merged/', 'Validation/원천데이터_상의_25/', 'Validation/라벨링데이터_상의_25/', '상의', 25)
extractData('Training/원천데이터_merged/', 'Training/라벨링데이터_merged/', 'Training/원천데이터_하의_100/', 'Training/라벨링데이터_하의_100/', '하의', 100)
extractData('Validation/원천데이터_merged/', 'Validation/라벨링데이터_merged/', 'Validation/원천데이터_하의_25/', 'Validation/라벨링데이터_하의_25/', '하의', 25)

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0] 스트리트_1 (23596)
[0, 0, 0, 2, 0, 0, 0, 0, 0, 0] 페미닌_k89sk2011-k89sk2011-s_4
[0, 0, 0, 2, 1, 0, 0, 0, 0, 0] 스트리트_gjkrle_ (14459)
[0, 0, 0, 3, 1, 0, 0, 0, 0, 0] 스트리트_20191203(중국,샘플)-bag042k9c(1)-오렌지(샘플,413헤링본에코)1203얼굴컷-밝기조정해주세요-IMG_1464
[0, 0, 0, 4, 1, 0, 0, 0, 0, 0] 모던_t4960k09-리디아(트윙클t)0911얼굴컷,몸매보정(팔뚝과일자허리보정,9번예시로작업)-13-IMG_2024
[0, 0, 0, 4, 1, 0, 0, 1, 0, 0] 페미닌_없음-1539-1539-sk1496(4)-엘리드레스(대행,펄주름sk)4.17-7-IMG_1622
[0, 0, 0, 4, 1, 1, 0, 1, 0, 0] 페미닌_sk1749-sk1749-s_6
[1, 0, 0, 4, 1, 1, 0, 1, 0, 0] 스트리트_gjkrle_ (7934)
[1, 0, 1, 4, 1, 1, 0, 1, 0, 0] 스트리트_20201117_232243_도르체8부-159376777270858300_1540053009
[1, 0, 1, 5, 1, 1, 0, 1, 0, 0] 리조트_pt3764k96-pt3764k96-s_4
[1, 0, 1, 5, 1, 2, 0, 1, 0, 0] 스트리트_Labelle_165_06
[1, 0, 1, 6, 1, 2, 0, 1, 0, 0] 스트리트_20180223(샘플)-t2309(4)-뽀글이(샘플,플라스틱)-10-IMG_4258
[1, 0, 1, 6, 1, 2, 0, 2, 0, 0] 스트리트_nb1009k01-nb1009k01-s_1
[1, 1, 1, 6, 1, 2, 0, 2, 0, 0] 페미닌_20200218(중국,샘플)-bl2853k02(4)-플라스틱(샘플,베베스트링)0218얼굴컷-밝기보정-IMG_8010
[1, 1,

[13, 5, 23, 24, 6, 26, 3, 11, 0, 3] 모던_2018-09-14 in calli 17836
[13, 5, 24, 24, 6, 26, 3, 11, 0, 3] 힙합_Sunnylike_210_09
[13, 5, 25, 24, 6, 26, 3, 11, 0, 3] 스트리트_20180511(샘플,중국)-t2448(4)-원포인트(샘플,비긴어게인T)-1-COB00778
[14, 5, 25, 24, 6, 26, 3, 11, 0, 3] 스트리트_ns192k02-ns192k02-s_1
[14, 5, 26, 24, 6, 26, 3, 11, 0, 3] 스트리트_t4336k06-t4336k06-s_1
[14, 5, 26, 24, 6, 27, 3, 11, 0, 3] 리조트_20200922sfd-1 (2789)
[14, 5, 26, 24, 6, 28, 3, 11, 0, 3] 젠더리스_20190917-nt1560k99(3)-커피샤워(대행,투포켓반PL)9.18-얼굴컷 사진 셀렉 신경써주시고 제품컷 색상 맞춰서 작업해주세여~-DHP06893
[14, 5, 26, 25, 6, 28, 3, 11, 0, 3] 스트리트_20201117_233335_쮸리부츠컷바지-159782921637240600_1909588730
[14, 5, 26, 26, 6, 28, 3, 11, 0, 3] 스트리트_20180117(중국)-t2270(4)-S.F(샘플,두겹데끼T)-9-IMG_8968
[14, 5, 26, 26, 6, 29, 3, 11, 0, 3] 스트리트_nt1894k01-nt1894k01-s_8
[14, 6, 26, 26, 6, 29, 3, 11, 0, 3] 모던_dongwon_035_09
[14, 6, 26, 26, 6, 29, 3, 12, 0, 3] 모던_F_RINA_289_02
[14, 6, 26, 27, 6, 29, 3, 12, 0, 3] 키치_160066999290612200_716120860
[15, 6, 26, 27, 6, 29, 3, 12, 0, 3] 스트리트_k87t255

[21, 17, 51, 49, 9, 44, 7, 17, 0, 5] 스트리트_2017_10 촬영원본-반-리ok-없음-6873-6873-해오라기-output-23-t2115(4)-아일렛(대행,R배색T) 10.12-02-IMG_9065
[21, 17, 51, 50, 9, 44, 7, 17, 0, 5] 스트리트_rldskfj_55 (1174)
[21, 17, 51, 50, 9, 45, 7, 17, 0, 5] 컨트리_nt1228k8b-nt1228k8b-s_8
[21, 17, 51, 50, 9, 45, 7, 18, 0, 5] 스트리트_J-tory_555_10
[22, 17, 51, 50, 9, 45, 7, 18, 0, 5] 리조트_laskdmci_01 (6688)
[22, 17, 51, 51, 9, 45, 7, 18, 0, 5] 스트리트_HONGSTAR_034_04
[22, 17, 51, 51, 9, 46, 7, 18, 0, 5] 모던_rldskfj_55 (2141)
[23, 17, 51, 51, 9, 46, 7, 18, 0, 5] 스트리트_gjkrle_ (7718)
[23, 18, 51, 51, 9, 46, 7, 18, 0, 5] 톰보이_할당완료-t2793k8b(4)-빈(샘플,슬폴라크롭)11.21-1-DSC_3932
[23, 18, 51, 51, 9, 47, 7, 18, 0, 5] 스트리트_없음-499-499-완-DSC_3253
[23, 19, 51, 51, 9, 47, 7, 18, 0, 5] 힙합_26-(23)-t1781(2)-송(대행,슈퍼MTM) 3.20-모델1-IMG_8206
[23, 19, 52, 51, 9, 47, 7, 18, 0, 5] 스트리트_Tank_268_08
[23, 19, 52, 51, 9, 48, 7, 18, 0, 5] 스트리트_2017_10 촬영원본-반-리ok-없음-6873-6873-해오라기-output-26-pt2112(1)-스테이진(대행,904청스판일자) 10.30-05-IMG_2303
[23, 19, 52, 52, 9, 48, 7, 18, 

[37, 23, 73, 73, 12, 75, 8, 29, 0, 7] 스트리트_2020_04 촬영원본-반-리-나스OK-3-(3424)-d12
[37, 23, 73, 73, 12, 76, 8, 29, 0, 7] 모던_1605237573155300_1 (1275)-1
[37, 23, 73, 73, 12, 76, 8, 30, 0, 7] 페미닌_20190920-nb970k99(2)-하바나(대행,실크와이드)9.23-얼굴컷 사진 셀렉 신경써주시고 제품컷 색상 맞춰서 작업해주세여~-DHP02890
[37, 23, 74, 73, 12, 76, 8, 30, 0, 7] 페미닌_1-(9)-006-273-2,029-9,543-PURPLE_049_06
[37, 23, 74, 74, 12, 76, 8, 30, 0, 7] 리조트_IMG_7626
[37, 23, 75, 74, 12, 76, 8, 30, 0, 7] 스트리트_lwerkjdjc_2 (1817)
[37, 23, 75, 74, 13, 76, 8, 30, 0, 7] 스트리트_2020_04 촬영원본-반-리-나스OK-5-(2129)-2-1
[37, 24, 75, 74, 13, 76, 8, 30, 0, 7] 스트리트_cvjvdlf_9 (2342)
[37, 24, 75, 74, 13, 77, 8, 30, 0, 7] 페미닌_20200117-cd868k01(3)-얼로우미(대행,더블버튼cd)-얼굴컷 사진 셀렉 신경써주시고 제품컷 색상 맞춰서 작업해주세여~-DHP09431
[37, 25, 75, 74, 13, 77, 8, 30, 0, 7] 스트리트_없음-386-386-output-k87cd454(4)-이비자(대행,미니Y)7.26-보정완료-DSC00536
[37, 25, 75, 74, 13, 77, 8, 30, 0, 8] 스트리트_t3693k01_s_16
[37, 25, 75, 75, 13, 77, 8, 30, 0, 8] 스트리트_Tdkfjdlkj_1 (4299)
[37, 25, 75, 75, 13, 77, 8, 31, 0, 8] 스트리트_20201

[43, 28, 99, 92, 17, 95, 12, 48, 0, 10] 소피스트케이티드_팰리스-vt257k99(4)-팰리스(대행,양모후드조끼)9.10-예쁘게 보정부탁드려욥(눈썹위치가 짝짝이라서 그것도...보정좀...부탁드려요...)-LJ8A1927
[43, 28, 99, 92, 17, 95, 12, 48, 0, 11] 스트리트_20191105-pt4206k9b(1)-투웨니식스(대행,P9481)11.06-얼굴컷 사진 셀렉 신경써주시고 제품컷 색상 맞춰서 작업해주세여~-DHP04830
[43, 28, 99, 92, 17, 95, 12, 49, 0, 11] 톰보이_Amaker_007_11
[43, 28, 100, 92, 17, 95, 12, 49, 0, 11] 모던_pt4634k02-600_1
[43, 28, 100, 93, 17, 95, 12, 49, 0, 11] 스트리트_2차-20180105(대행)21-sk1777(3)-팝콘(대행,쭈리시보리SK)1.9-04-IMG_0404
[43, 28, 100, 94, 17, 95, 12, 49, 0, 11] 스트리트_20200218(중국,샘플)-sk2565k02(8)-오빠네(샘플,소프트롱치마)0218얼굴컷-밝기보정해주세요-IMG_9319
[43, 28, 100, 94, 17, 96, 12, 49, 0, 11] 리조트_siutim_1223_36
[43, 28, 100, 95, 17, 96, 12, 49, 0, 11] 리조트_set585k8b(4)-퍼니펑크(대행,밍크세트)12.5-입술 다문컷만 입술 살려서 작업해주세요-DSC_8899
[43, 28, 100, 96, 17, 96, 12, 49, 0, 11] 스트리트_1107-14428-14428 (2-1)-20200129-1 (207)
[43, 28, 100, 96, 17, 96, 12, 49, 0, 12] 스포티_20180326(샘플)--2001466 (1813)
[43, 29, 100, 96, 17, 96, 12, 49, 0, 12] 스트리트_pt4117k99-pt4117k9

...

[2, 17, 3, 20, 0, 0] 스트리트_gjkrle_ (821)
[2, 17, 3, 21, 0, 0] 젠더리스_1103 신규작업-4-output-2-(637)-1
[2, 17, 3, 22, 0, 0] 스트리트_1 (25550)
[2, 17, 4, 22, 0, 0] 톰보이_DAKS_1597_08
[2, 17, 4, 23, 0, 0] 스트리트_bl2116k92-bl2116k92-s_8
[2, 18, 4, 23, 0, 0] 스트리트_LIEGE_035_05
[2, 19, 4, 23, 0, 0] 스트리트_20180207(중국)-bl1060(1)-중국사입(812#)모델컷변경-09-IMG_0934
[2, 19, 5, 23, 0, 0] 리조트_bl863-bl863-s_04
[2, 19, 5, 24, 0, 0] 톰보이_20191126(중국,샘플)-t3558k9b(6)-쥬디스(샘플,실버벨mtm)1126얼굴컷(핑크색디테일컷물묻은거지워주세요)-밝기보정해주세요-IMG_5837
[2, 19, 5, 25, 0, 0] 스트리트_20201117_000444_찰리통-158615683981211500_348972493
[2, 19, 5, 26, 0, 0] 스트리트_pt5020k05-pt5020k05-s_10
[2, 19, 5, 27, 0, 0] 매니시_Mimo_140_02
[2, 20, 5, 27, 0, 0] 스트리트_20191008(중국,샘플)-bl2711k9a(2)-아가씨(샘플,숀NB)1008얼굴컷-밝기보정해주세여-IMG_9661
[2, 20, 6, 27, 0, 0] 스트리트_없음-1539-1539-set173(2)-중국(5446#) 4.20-5-IMG_6308
[2, 20, 6, 28, 0, 0] 스트리트_20190502(대행)-할당-제품컷있음-bl2382k94(4)-바오바오(대행,라미NB)05.02-장주연님-1-DSC01627
[2, 21, 6, 28, 0, 0] 스트리트_2017_10 촬영원본-반-리ok-없음-13835-13835-lkt_1 (6136)
[2, 21, 6, 29

[9, 61, 21, 83, 0, 1] 모던_k89bl1828-k89bl1828-s_3
[9, 62, 21, 83, 0, 1] 스트리트_k87pt2743-k87pt2743-s_5 (1)
[9, 62, 22, 83, 0, 1] 키치_20190513(대행)-할당-t3157k95(2)지샵걸(대행,루피T)5.14-1-IMG_3082
[9, 62, 22, 84, 0, 1] 리조트_20191121-재촬 nt1638k9a(4)-팰리스(대행,시크RT)10.22-얼굴컷 사진 셀렉 신경써주시고 제품컷 색상 맞춰서 작업해주세여~-DHP05119
[9, 62, 22, 85, 0, 1] 레트로_없음-273-273-output-k87bl1709(3)-MONO+(샘플,도트나시OPS)-1-IMG_9219
[9, 62, 22, 86, 0, 1] 리조트_2014-06-09 in calli 52973
[9, 62, 22, 87, 0, 1] 모던_Recipe_128_28
[9, 62, 22, 88, 0, 1] 스트리트_20180518(대행)19-nt950(3)-커피샤워(대행,리프나시)5.23-화이트-DSC00745
[9, 62, 22, 89, 0, 1] 리조트_REIGN_009_05
[9, 63, 22, 89, 0, 1] 스트리트_Pinocchio_023_08
[9, 63, 22, 90, 0, 1] 소피스트케이티드_BOGGU_002_11
[9, 63, 22, 91, 0, 1] 컨트리_2019-05-24 sw.Jang303184
[9, 63, 22, 92, 0, 1] 리조트_1-(9)-006-273-2,029-9,543-PURPLE_096_02
[9, 64, 22, 92, 0, 1] 스트리트_gjkrle_ (5742)
[9, 64, 22, 93, 0, 1] 컨트리_Haemil_304_57
[9, 64, 22, 94, 0, 1] 모던_2-output-2-(10)-20190215(대행)-cd600k92(4)-팰리스(대행,단작가디건)2.15-1-DSC_5187
[9, 64, 22, 95, 0, 1] 밀

[38, 100, 62, 100, 0, 7] 젠더리스_iljuk_119_01
[38, 100, 63, 100, 0, 7] 스포티_20190315(샘플,중국)-최정임님-set610k93(2)-원포인트(샘플,반바지SET)0315-모델-IMG_3676
[38, 100, 64, 100, 0, 7] 페미닌_1-(9)-005-272-2,351-7,514-FisrtGI_365_05
[38, 100, 65, 100, 0, 7] 소피스트케이티드_없음-3198-3198-201-1 (3085)
[38, 100, 66, 100, 0, 7] 리조트_id_727_05
[38, 100, 66, 100, 0, 8] 스트리트_2020_04 촬영원본-반-리-나스OK-1-(1753)-1
[38, 100, 67, 100, 0, 8] 스트리트_없음-1490-1490-bl678(2)-엘리드레스(대행,리본st남방)4.17-3-IMG_9772
[38, 100, 68, 100, 0, 8] 스트리트_t4464k07-t4464k07-s_7
[38, 100, 69, 100, 0, 8] 오리엔탈_u_156222381244102800_323028766
[38, 100, 70, 100, 0, 8] 스트리트_2020_04 촬영원본-반-리-나스OK-3-(2512)-d12
[38, 100, 70, 100, 0, 9] 스포티_1 (30511)
[38, 100, 70, 100, 0, 10] 스트리트_1603076-0001 (223)
[38, 100, 71, 100, 0, 10] 스트리트_20190628(중국,샘플)-t3258k96(4)-소소(샘플,문크롭T)0628얼굴컷-라이트룸-IMG_2800
[38, 100, 72, 100, 0, 10] 모던_pt1712(2)-젤리(대행,스텔라SL) 4.4-2-IMG_6414
[39, 100, 72, 100, 0, 10] 섹시_bl3140c05-s_12
[39, 100, 73, 100, 0, 10] 모던_pt1734(4)-투엘(대행,마랩pt)4.13-2-IMG_2788
[39, 100, 

[100, 100, 100, 100, 1, 38] 스트리트_pt5336c06-pt5336c06-s_15
[100, 100, 100, 100, 1, 39] 스트리트_20180525(대행)24-pt2567(1)-판도라(대행,스파이더1)5.29-1-DSC01128
[100, 100, 100, 100, 1, 40] 스트리트_20201118_000558_기모배색조거-160431083617067000_112502553
[100, 100, 100, 100, 1, 41] 힙합_pt6022k09-pt6022k09-s_6
[100, 100, 100, 100, 1, 42] 스포티_20201117_001520_이선시보리팬츠-158813625163792900_1172577107
[100, 100, 100, 100, 1, 43] 모던_byhoney_048_03
[100, 100, 100, 100, 1, 44] 스트리트_20201117_233840_아노락셋업팬츠-159885538697091200_237385483
[100, 100, 100, 100, 1, 45] 톰보이_20201118_000443_기모크롭스티치-160398802696388000_1920845301
[100, 100, 100, 100, 1, 46] 스트리트_159850711348862500_973810591
[100, 100, 100, 100, 1, 47] 모던_20180117(대행)26-pt2222(2)-아론(대행,통바지SL)1.23-27-IMG_9810
[100, 100, 100, 100, 1, 48] 스트리트_20201117_235546_264번 골덴조거-160068087918918200_2104152303
[100, 100, 100, 100, 1, 49] 스포티_122-122-set980k09-set980k09-s_2
[100, 100, 100, 100, 1, 50] 스트리트_20201118_000308_수면바지-160325982428913499_979840953
[100, 100, 100, 100, 1, 51] 

[0, 10, 4, 7, 0, 0] 스트리트_pt4523k02-pt4523k02-s_1
[0, 11, 4, 7, 0, 0] 스트리트_20190726(중국,샘플)-t3318k97(7)-소소(샘플,베이직단가라)0726얼굴컷-라이트룸-IMG_9275
[0, 12, 4, 7, 0, 0] 스트리트_3-output-2-(10)-20190215(샘플)-할당-pt3394k92(1)-이피아이진(샘플,918)02.14-최정임님-1-IMG_6426
[0, 13, 4, 7, 0, 0] 스트리트_20190709(중국,샘플)-t3283k97(3)-바비제이(샘플,마틴크롭)0709얼굴컷-15-IMG_5757
[0, 13, 4, 8, 0, 0] 페미닌_80426(대행)24-bl13-2080 (632)
[0, 14, 4, 8, 0, 0] 스트리트_2017_10 촬영원본-반-리ok-없음-13835-13835-lkt_1 (1028)
[0, 15, 4, 8, 0, 0] 스트리트_nb1032k02-s_7
[0, 15, 4, 9, 0, 0] 모던_10.30-bl609(2)-다다(대행,모노BL)3.29-5-IMG_4030
[0, 15, 4, 10, 0, 0] 리조트_gjkrle_ (3443)
[0, 15, 4, 11, 0, 0] 클래식_20191107(중국,샘플)-jk1984k9b(5)-지끼(샘플,로이코트)1107얼굴컷-라이트룸-IMG_0298
[0, 15, 4, 12, 0, 0] 스트리트_u_15728639925921800_1798211774
[0, 16, 4, 12, 0, 0] 스트리트_20190328(대행)-할당-pt3514k93(1)-UB40(대행,214_연청밴딩부츠컷)03.28-행거-1-DSC02609
[0, 16, 4, 13, 0, 0] 매니시_2-2-7494-14988-2-2-20190123(대행)-할당-pt3322k91(3)-엘투(대행,515)1.22-앉아있는사진은 페이지작업만 해주시고 S파일은 무조건 주름없는사진으로 셀렉바랍니다-DSC_0736
[0, 17, 4, 13, 0, 0] 로맨

In [140]:
img_dir = 'Trainig/원천데이터_all_in_one/'
label_dir = 'Training/라벨링데이터_all_in_one/'
res_img_dir = 'Training/원천데이터_상의_1000/'
res_label_dir = 'Training/라벨링데이터_상의_1000/'
category_type = '상의'
npc = 100 # number per category

#extractData(img_dir, label_dir, res_img_dir, res_label_dir, category_type, npc)

In [25]:
print(os.path.isfile('Trainig/원천데이터_merged/스트리트_wq(278).jpg'))
print(os.path.isdir('Trainig/'))

False
False
